In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from wikipediaapi import Wikipedia
import plotly.graph_objects as go
import numpy as np

movies = pd.read_csv('data/all_movies_with_success_index.csv')
actors = pd.read_csv('data/actor_with_scoreindex.csv')

actors

,Actor name,Cumulative Score,Actor Score Index
0,'squeeks'_the_caterpillar,1.779819,4.116636
1,40_glocc,1.393875,2.817460
2,50_cent,1.776826,4.106561
3,a._delon_ellis_jr.,1.676044,3.767307
4,a._j._benza,1.821579,4.257209
...,...,...,...
20688,éva_igó,1.484994,3.124189
20689,íñigo_garcés,1.622983,3.588689
20690,óscar_jaenada,1.648598,3.674915
20691,željko_ivanek,2.118027,5.255119


In [33]:
actors['Actor Score Index'].describe()

count    20693.000000
mean         3.884397
std          0.729746
min          0.000000
25%          3.520722
50%          3.890783
75%          4.266474
max         10.000000
Name: Actor Score Index, dtype: float64

In [65]:
actors.sort_values(by='Actor Score Index', ascending=False, inplace=True)

percentage = 0.25
top_actors = actors.iloc[:int(percentage*len(actors))]
worst_actors = actors.iloc[-int(percentage*len(actors)):]

top_actors

,Actor name,Cumulative Score,Actor Score Index
4946,denzel_washington,3.527584,10.000000
13290,matt_damon,3.435967,9.691597
19440,tom_hanks,3.274563,9.148275
6445,frank_welker,3.253477,9.077296
5535,eddie_murphy,3.205583,8.916074
...,...,...,...
17563,sarah_clarke,1.824422,4.266780
19225,tilly_mandelbrot,1.824332,4.266474
4299,danielle_darrieux,1.824332,4.266474
722,amethyste_frezignac,1.824332,4.266474


In [78]:
fig = go.Figure()

# Add histogram for top actors
fig.add_trace(go.Histogram(
    x=actors['Actor Score Index'],
    nbinsx=500,
    name='Top Actors',
    opacity=0.5,
    marker_color='pink'
))

fig.update_layout(
    title='Distribution of actor sucess index',
    xaxis_title='Actor Score Index',
    yaxis_title='Frequency',
    barmode='overlay',
    legend=dict(x=0.8, y=1),
    template='plotly_white'
)

# Show plot
fig.show()

In [76]:
# plot the distribution of the 25% worst and best actors
 
fig = go.Figure()

# Add histogram for top actors
fig.add_trace(go.Histogram(
    x=top_actors['Actor Score Index'],
    nbinsx=100,
    name='Top Actors',
    opacity=0.5,
    marker_color='green'
))

# Add histogram for worst actors
fig.add_trace(go.Histogram(
    x=worst_actors['Actor Score Index'],
    nbinsx=100,
    name='Worst Actors',
    opacity=0.5,
    marker_color='red'
))

fig.update_layout(
    title='Distribution of actor sucess index',
    xaxis_title='Actor Score Index',
    yaxis_title='Frequency',
    barmode='overlay',
    legend=dict(x=0.8, y=1),
    template='plotly_white'
)

# Show plot
fig.show()


In [100]:
# in movie find the top actors movies

# Best actors data
top_actors_names = top_actors['Actor name'].values.tolist()
top_movies = movies[movies['Actor name'].isin(top_actors_names)]
top_companies_best = top_movies['Movie company'].value_counts().head(15)

# Worst actors data
worst_actors_names = worst_actors['Actor name'].values.tolist()
worst_movies = movies[movies['Actor name'].isin(worst_actors_names)]
top_companies_worst = worst_movies['Movie company'].value_counts().head(15)

# Create subplots with two pie charts
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                    subplot_titles=['Top 15 Best Actors Movie Companies', 'Top 15 Worst Actors Movie Companies'])

# Add the pie chart for best actors
fig.add_trace(go.Pie(
    labels=top_companies_best.index,
    values=top_companies_best.values,
    hole=0.3,
    name='Best Actors'
), row=1, col=1)

# Add the pie chart for worst actors
fig.add_trace(go.Pie(
    labels=top_companies_worst.index,
    values=top_companies_worst.values,
    hole=0.3,
    name='Worst Actors'
), row=1, col=2)

# Update layout
fig.update_layout(
    title_text='Comparison of Movie Companies for Best and Worst Actors',
    template='plotly_white'
)

fig.show()

NameError: name 'make_subplots' is not defined

## Scrapping nationality 


In [38]:
actors_to_scrap = pd.read_csv('data/actor_unique.csv')


USER_AGENT = "ADAProject (example@epfl.ch)"
# Initialize Wikipedia API
wiki_wiki = Wikipedia(
    language='en', 
    user_agent=USER_AGENT,
    headers={'User-Agent': USER_AGENT}
)



In [47]:
def fetch_actor_summary(actor_name):
    try:
        page = wiki_wiki.page(actor_name)
        if page.exists():
            return page.summary  # Fetch the full summary
    except Exception as e:
        print(f"Error fetching summary for {actor_name}: {e}")
    return None


import re
def extract_nationality_after_born(sentence):
    # Handle None input
    if sentence is None:
        return None

    # Locate the first occurrence of "born"
    born_index = sentence.lower().find("born")
    if born_index == -1:
        return None  # Return None if "born" is not found

    # Extract the 300 characters after "born" for more context
    text_after_born = sentence[born_index:born_index + 300]

    # Regex pattern to find "[nationality] actor" or similar
    match = re.search(r"\b([A-Za-z]+)\s(?:actor|actress|director|music producer|singer|rapper|film|journalist|gossip columnist|playwright|film director|acting|multi-award winning|radio personality|comedian|radio personality|professional wrestler|comedian|voice actor|former actor|screenwriter|artist|television personality|music executive)", text_after_born, re.IGNORECASE)
    if match:
        return match.group(1).capitalize()  # Return the nationality, capitalized

    # Fallback: Look for standalone nationality (e.g., "American") near "actor"
    fallback_match = re.search(r"\b(American|English|Australian|Canadian|British|French|Italian|German|Spanish)\b", text_after_born, re.IGNORECASE)
    if fallback_match:
        return fallback_match.group(1).capitalize()

    # Final fallback: Use the last word before "actor" if no other nationality is found
    pre_actor_match = re.search(r"([A-Za-z]+)\sactor", text_after_born, re.IGNORECASE)
    if pre_actor_match:
        return pre_actor_match.group(1).capitalize()

    return None  # Return None if no match is found


# Update the processing loop to use the improved extraction
list_nationality = []
not_found = 0 

for actor in actors:
    nationality_info = fetch_actor_summary(actor)  # Fetch the summary
    if nationality_info:
        nationality = extract_nationality_after_born(nationality_info)
        if nationality: 
            list_nationality.append(nationality)
            print(f"Actor: {actor}\nExtracted Nationality: {nationality}\n")
        else:
            print(f"Actor: {actor}\nNationality Info: {nationality_info[:200]}\nExtracted Nationality: None\n")
    else:
        not_found += 1
        print(f"Actor: {actor}\nNationality Info: Not found\nExtracted Nationality: Not found\n")

print(f"Number of actors not found: {not_found}")

Actor: 'Squeeks' The Caterpillar
Nationality Info: Not found
Extracted Nationality: Not found

Actor: 40 Glocc
Extracted Nationality: American

Actor: 50 Cent
Extracted Nationality: American

Actor: A. Delon Ellis Jr.
Nationality Info: Not found
Extracted Nationality: Not found

Actor: A. J. Benza
Nationality Info: Alfred Joseph Benza is an American gossip columnist and television host.
Extracted Nationality: None

Actor: A. J. Bowen
Extracted Nationality: American

Actor: A. J. Cook
Extracted Nationality: Canadian

Actor: A. J. Langer
Extracted Nationality: American

Actor: A. Jay Radcliff
Nationality Info: Not found
Extracted Nationality: Not found

Actor: A. Russell Andrews
Nationality Info: Not found
Extracted Nationality: Not found

Actor: A.D. Miles
Extracted Nationality: American

Actor: Aaliyah
Nationality Info: Aaliyah Dana Haughton ( ah-LEE-ə; January 16, 1979 – August 25, 2001), known as Aaliyah, was an American singer, actress, dancer, and model. She has been credited with 

KeyboardInterrupt: 